# 02 Rolling Forecast Factory

Generate rolling window forecasts for all models: TabPFN-TS, Chronos-Tiny, SARIMA+Fourier, and LightGBM.

**Core Configuration:**
- Horizons: 1, 7, 14, 28 days ahead
- Origins: Daily from 2024-01-01 to 2025-06-01
- Minimum training window: 730 days (2 years)

**Critical:** All train/test splits use date slicing only (never iloc) to prevent leakage.

## 0. Setup

### Imports

In [43]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [44]:
# Model imports
import statsmodels.api as sm
from statsmodels.tsa.deterministic import DeterministicProcess
from autogluon.timeseries import TimeSeriesDataFrame
from tabpfn_time_series import TabPFNTimeSeriesPredictor, TabPFNMode
from chronos import ChronosPipeline, BaseChronosPipeline
import lightgbm as lgb

In [45]:
# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

In [46]:
# Define paths
project_root = Path.cwd().parent
data_dir = project_root / 'data'
results_dir = project_root / 'results' / 'forecasts'
results_dir.mkdir(parents=True, exist_ok=True)

### Load Data

In [47]:
# Load cleaned time series
ts = pd.read_pickle(data_dir / 'flu_daily_clean.pkl')
print(f"Loaded data: {ts.shape[0]} observations")
print(f"Date range: {ts.index.min()} to {ts.index.max()}")
print(f"Frequency: {ts.index.freq}")

Loaded data: 1078 observations
Date range: 2022-07-04 00:00:00 to 2025-06-15 00:00:00
Frequency: <Day>


### Configure Rolling Windows

In [48]:
# Forecast configuration - OPTIMIZED for API efficiency
HORIZONS = [7, 28]  # Short-term (weekly) vs long-term (monthly) forecasts
# Bi-weekly origins = 24 forecast points (statistically robust, API-friendly)
ORIGINS = pd.date_range('2024-07-08', '2025-05-26', freq='2W-MON')
MIN_TRAIN = 730  # 2 years minimum training data

print(f"Forecast horizons: {HORIZONS}")
print(f"Number of forecast origins: {len(ORIGINS)} (bi-weekly)")
print(f"Minimum training days: {MIN_TRAIN}")
print(f"Total forecasts per model: {len(ORIGINS) * len(HORIZONS)}")
print(f"\n✅ Optimized for API efficiency: 48 calls per model")
print(f"   (Covers 11 months, 24 time points, 2 horizons)")

Forecast horizons: [7, 28]
Number of forecast origins: 24 (bi-weekly)
Minimum training days: 730
Total forecasts per model: 48

✅ Optimized for API efficiency: 48 calls per model
   (Covers 11 months, 24 time points, 2 horizons)


### Initialize TabPFN Client

In [49]:
from dotenv import load_dotenv
import os
import tabpfn_client

# Load API key from .env
env_file = project_root / ".env"
if env_file.exists():
    load_dotenv(env_file)
    api_key = os.getenv("PRIORLABS_API_KEY")
    if api_key:
        # CRITICAL: Use set_access_token (NOT init with api_key parameter)
        tabpfn_client.set_access_token(api_key)
        print("✅ TabPFN client initialized")
    else:
        print("⚠️  PRIORLABS_API_KEY not found in .env")
else:
    print("⚠️  .env file not found")

✅ TabPFN client initialized


### Initialize Chronos Model

In [50]:
# Suppress transformers/huggingface progress bar warnings
import os
os.environ['HF_HUB_DISABLE_PROGRESS_BARS'] = '1'

# Load Chronos-Tiny model (CPU-only)
print("Loading Chronos-Tiny model (this may take 10-15 seconds)...")
chronos_pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-tiny",
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)
print("✅ Chronos-Tiny model loaded (CPU)")

Loading Chronos-Tiny model (this may take 10-15 seconds)...
✅ Chronos-Tiny model loaded (CPU)


## 1. Helper Functions

In [51]:
def build_fourier_terms(dates, period=365, order=2):
    """
    Build Fourier terms for seasonality from date index.

    Parameters
    ----------
    dates : pd.DatetimeIndex
        Date index to compute Fourier terms for
    period : int
        Seasonal period (365 for annual cycle)
    order : int
        Number of sine/cosine pairs (order=2 gives 4 terms)

    Returns
    -------
    pd.DataFrame
        Fourier terms with columns sin1, cos1, sin2, cos2, ...
    """
    fourier = pd.DataFrame(index=dates)
    for k in range(1, order + 1):
        fourier[f'sin{k}'] = np.sin(2 * np.pi * k * np.arange(len(dates)) / period)
        fourier[f'cos{k}'] = np.cos(2 * np.pi * k * np.arange(len(dates)) / period)
    return fourier

In [52]:
def build_lag_features(series, lags=[1, 2, 3, 7, 14]):
    """
    Build lag features from a time series.

    Parameters
    ----------
    series : pd.Series
        Time series to create lags from
    lags : list of int
        Lag values to create

    Returns
    -------
    pd.DataFrame
        Dataframe with lag columns
    """
    lag_df = pd.DataFrame(index=series.index)
    for lag in lags:
        lag_df[f'lag_{lag}'] = series.shift(lag)
    return lag_df

## 2. Model: SARIMA + Fourier

In [53]:
def forecast_sarima_fourier(train_series, horizon, order=(1, 0, 1), fourier_order=2, period=365):
    """
    Forecast with SARIMA + Fourier terms for seasonality.

    Parameters
    ----------
    train_series : pd.Series
        Training data (datetime-indexed)
    horizon : int
        Number of steps ahead to forecast
    order : tuple
        ARIMA order (p, d, q)
    fourier_order : int
        Number of Fourier term pairs
    period : int
        Seasonal period for Fourier terms

    Returns
    -------
    dict : {'q0.1': float, 'q0.5': float, 'q0.9': float}
        Forecast quantiles
    """
    # Create Fourier terms for train period
    fourier_train = build_fourier_terms(train_series.index, period=period, order=fourier_order)

    # Create Fourier terms for forecast period
    forecast_start = train_series.index[-1] + pd.Timedelta(days=1)
    forecast_dates = pd.date_range(forecast_start, periods=horizon, freq='D')
    fourier_test = build_fourier_terms(forecast_dates, period=period, order=fourier_order)

    # Fit SARIMA with Fourier exogenous variables
    model = sm.tsa.SARIMAX(
        train_series,
        order=order,
        seasonal_order=(0, 0, 0, 0),  # No seasonal ARIMA (use Fourier instead)
        trend='c',
        exog=fourier_train
    )

    try:
        fit = model.fit(disp=False, maxiter=200)

        # Point forecast
        forecast = fit.forecast(steps=horizon, exog=fourier_test)
        point = forecast.iloc[-1]

        # Get forecast uncertainty (use standard error for intervals)
        forecast_obj = fit.get_forecast(steps=horizon, exog=fourier_test)
        pred_int = forecast_obj.conf_int(alpha=0.2)  # 80% interval

        return {
            'q0.1': float(pred_int.iloc[-1, 0]),  # lower bound
            'q0.5': float(point),  # median (point forecast)
            'q0.9': float(pred_int.iloc[-1, 1])   # upper bound
        }
    except Exception as e:
        # If fitting fails, return naive forecast
        naive = float(train_series.iloc[-1])
        return {'q0.1': naive * 0.8, 'q0.5': naive, 'q0.9': naive * 1.2}

### Test SARIMA + Fourier

In [54]:
# Test on a single rolling window
test_origin = pd.Timestamp('2024-06-01')
test_horizon = 7

# Get training data (all data up to origin)
train = ts[ts.index < test_origin]
print(f"Train size: {len(train)} days")
print(f"Train range: {train.index.min()} to {train.index.max()}")

# Generate forecast
pred = forecast_sarima_fourier(train, test_horizon)
print(f"\nForecast for {test_origin + pd.Timedelta(days=test_horizon - 1)}:")
print(f"  q0.1: {pred['q0.1']:.2f}%")
print(f"  q0.5: {pred['q0.5']:.2f}%")
print(f"  q0.9: {pred['q0.9']:.2f}%")

# Compare to actual
actual_date = test_origin + pd.Timedelta(days=test_horizon - 1)
if actual_date in ts.index:
    actual = ts.loc[actual_date]
    print(f"\nActual: {actual:.2f}%")
    print(f"Within 80% interval: {pred['q0.1'] <= actual <= pred['q0.9']}")

Train size: 698 days
Train range: 2022-07-04 00:00:00 to 2024-05-31 00:00:00

Forecast for 2024-06-07 00:00:00:
  q0.1: -0.09%
  q0.5: 1.16%
  q0.9: 2.41%

Actual: 0.90%
Within 80% interval: True


## 3. Model: LightGBM with Lags + Fourier

In [55]:
def forecast_lightgbm(train_series, horizon, lags=[1, 2, 3, 7, 14], fourier_order=2, period=365):
    """
    Forecast with LightGBM using lag features + Fourier terms.

    Parameters
    ----------
    train_series : pd.Series
        Training data (datetime-indexed)
    horizon : int
        Number of steps ahead to forecast
    lags : list of int
        Lag features to create
    fourier_order : int
        Number of Fourier term pairs
    period : int
        Seasonal period for Fourier terms

    Returns
    -------
    dict : {'q0.1': float, 'q0.5': float, 'q0.9': float}
        Forecast quantiles
    """
    # Build lag features
    lag_df = build_lag_features(train_series, lags=lags)

    # Build Fourier features
    fourier_df = build_fourier_terms(train_series.index, period=period, order=fourier_order)

    # Combine features
    X_train = pd.concat([lag_df, fourier_df], axis=1).dropna()
    y_train = train_series.loc[X_train.index]

    # Train LightGBM models for each quantile
    quantiles = [0.1, 0.5, 0.9]
    predictions = {}

    for q in quantiles:
        model = lgb.LGBMRegressor(
            objective='quantile',
            alpha=q,
            n_estimators=300,
            max_depth=5,
            learning_rate=0.05,
            random_state=42,
            verbose=-1
        )
        model.fit(X_train, y_train)

        # Multi-step forecast (iterative)
        current_series = train_series.copy()

        for step in range(horizon):
            # Build features for next step
            lag_feats = build_lag_features(current_series, lags=lags).iloc[-1:]

            # Fourier terms for next date
            next_date = current_series.index[-1] + pd.Timedelta(days=1)
            fourier_feats = build_fourier_terms(pd.DatetimeIndex([next_date]), period=period, order=fourier_order)

            # Combine and predict
            X_next = pd.concat([lag_feats, fourier_feats], axis=1)
            pred = model.predict(X_next)[0]

            # Append prediction to series for next iteration
            current_series = pd.concat([
                current_series,
                pd.Series([pred], index=[next_date])
            ])

        # Store final prediction
        predictions[f'q{q}'] = float(pred)

    # Enforce quantile monotonicity: q0.1 ≤ q0.5 ≤ q0.9
    # This fixes the quantile crossing issue in LightGBM quantile regression
    predictions['q0.1'] = min(predictions['q0.1'], predictions['q0.5'])
    predictions['q0.9'] = max(predictions['q0.5'], predictions['q0.9'])

    return predictions

### Test LightGBM

In [56]:
# Test LightGBM on same window
pred_lgb = forecast_lightgbm(train, test_horizon)
print(f"LightGBM forecast for {actual_date}:")
print(f"  q0.1: {pred_lgb['q0.1']:.2f}%")
print(f"  q0.5: {pred_lgb['q0.5']:.2f}%")
print(f"  q0.9: {pred_lgb['q0.9']:.2f}%")
print(f"\nActual: {actual:.2f}%")
print(f"Within 80% interval: {pred_lgb['q0.1'] <= actual <= pred_lgb['q0.9']}")

LightGBM forecast for 2024-06-07 00:00:00:
  q0.1: 0.70%
  q0.5: 1.02%
  q0.9: 1.45%

Actual: 0.90%
Within 80% interval: True


## 4. Model: TabPFN-TS

In [57]:
def forecast_tabpfn(train_series, horizon, item_id='flu_positivity'):
    """
    Forecast with TabPFN-TS (zero-shot foundation model).

    Parameters
    ----------
    train_series : pd.Series
        Training data (datetime-indexed)
    horizon : int
        Number of steps ahead to forecast
    item_id : str
        Identifier for the time series

    Returns
    -------
    dict : {'q0.1': float, 'q0.5': float, 'q0.9': float}
        Forecast quantiles
    """
    # Prepare training data as TimeSeriesDataFrame
    df_prep = train_series.reset_index()
    df_prep.columns = ['timestamp', 'target']
    df_prep['item_id'] = item_id

    # Add calendar features
    df_prep['day_of_year'] = df_prep['timestamp'].dt.dayofyear
    df_prep['month'] = df_prep['timestamp'].dt.month

    train_tsdf = TimeSeriesDataFrame.from_data_frame(
        df_prep[['item_id', 'timestamp', 'target', 'day_of_year', 'month']],
        id_column='item_id',
        timestamp_column='timestamp'
    )

    # Create test dates
    forecast_start = train_series.index[-1] + pd.Timedelta(days=1)
    test_dates = pd.date_range(forecast_start, periods=horizon, freq='D')

    test_df = pd.DataFrame({
        'timestamp': test_dates,
        'item_id': item_id,
        'target': np.nan,
        'day_of_year': test_dates.dayofyear,
        'month': test_dates.month
    })

    test_tsdf = TimeSeriesDataFrame.from_data_frame(
        test_df,
        id_column='item_id',
        timestamp_column='timestamp'
    )

    # Initialize predictor and forecast
    predictor = TabPFNTimeSeriesPredictor(tabpfn_mode=TabPFNMode.CLIENT)
    pred = predictor.predict(train_tsdf, test_tsdf)

    # Extract final horizon prediction
    pred_slice = pred.loc[item_id]

    return {
        'q0.1': float(pred_slice[0.1].iloc[-1]),
        'q0.5': float(pred_slice[0.5].iloc[-1]),
        'q0.9': float(pred_slice[0.9].iloc[-1])
    }

### Test TabPFN-TS

In [58]:
# Test TabPFN-TS on same window (warning: this makes API calls)
print("Testing TabPFN-TS (may take a few seconds)...")
pred_tabpfn = forecast_tabpfn(train, test_horizon)
print(f"\nTabPFN-TS forecast for {actual_date}:")
print(f"  q0.1: {pred_tabpfn['q0.1']:.2f}%")
print(f"  q0.5: {pred_tabpfn['q0.5']:.2f}%")
print(f"  q0.9: {pred_tabpfn['q0.9']:.2f}%")
print(f"\nActual: {actual:.2f}%")
print(f"Within 80% interval: {pred_tabpfn['q0.1'] <= actual <= pred_tabpfn['q0.9']}")

Testing TabPFN-TS (may take a few seconds)...


Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 696.84it/s]
Processing: 100%|██████████| [00:01<00:00]


TabPFN-TS forecast for 2024-06-07 00:00:00:
  q0.1: 0.40%
  q0.5: 0.51%
  q0.9: 0.60%

Actual: 0.90%
Within 80% interval: False


## 5. Model: Chronos-Tiny

In [59]:
def forecast_chronos(train_series, horizon, pipeline, num_samples=100):
    """
    Forecast with Chronos-Tiny (zero-shot foundation model).

    Parameters
    ----------
    train_series : pd.Series
        Training data (datetime-indexed)
    horizon : int
        Number of steps ahead to forecast
    pipeline : ChronosPipeline
        Pre-loaded Chronos model pipeline
    num_samples : int
        Number of forecast samples for quantile estimation

    Returns
    -------
    dict : {'q0.1': float, 'q0.5': float, 'q0.9': float}
        Forecast quantiles
    """
    # Convert to tensor
    context = torch.tensor(train_series.values, dtype=torch.float32)

    # Generate forecast samples
    forecast_samples = pipeline.predict(
        context=context.unsqueeze(0),  # Add batch dimension
        prediction_length=horizon,
        num_samples=num_samples
    )

    # Extract final horizon predictions and compute quantiles
    final_preds = forecast_samples[0, :, -1].numpy()  # [num_samples]

    return {
        'q0.1': float(np.quantile(final_preds, 0.1)),
        'q0.5': float(np.quantile(final_preds, 0.5)),
        'q0.9': float(np.quantile(final_preds, 0.9))
    }

### Test Chronos-Tiny

In [60]:
# Test Chronos-Tiny on same window
print("Testing Chronos-Tiny...")
pred_chronos = forecast_chronos(train, test_horizon, chronos_pipeline)
print(f"\nChronos-Tiny forecast for {actual_date}:")
print(f"  q0.1: {pred_chronos['q0.1']:.2f}%")
print(f"  q0.5: {pred_chronos['q0.5']:.2f}%")
print(f"  q0.9: {pred_chronos['q0.9']:.2f}%")
print(f"\nActual: {actual:.2f}%")
print(f"Within 80% interval: {pred_chronos['q0.1'] <= actual <= pred_chronos['q0.9']}")

Testing Chronos-Tiny...

Chronos-Tiny forecast for 2024-06-07 00:00:00:
  q0.1: 0.29%
  q0.5: 0.69%
  q0.9: 1.19%

Actual: 0.90%
Within 80% interval: True


## 6. Rolling Forecast Loop

**Critical:** Execute all models across all origins × horizons.

In [61]:
def run_rolling_forecasts(ts, origins, horizons, min_train):
    """
    Run rolling forecasts for all models.

    Parameters
    ----------
    ts : pd.Series
        Full time series
    origins : pd.DatetimeIndex
        Forecast origin dates
    horizons : list of int
        Forecast horizons
    min_train : int
        Minimum training window size

    Returns
    -------
    dict : {model_name: pd.DataFrame}
        Forecast results by model
    """
    results = {
        'sarima_fourier': [],
        'lightgbm': [],
        'tabpfn': [],
        'chronos': []
    }

    # Progress bar
    total_iter = len(origins) * len(horizons)
    pbar = tqdm(total=total_iter, desc="Rolling forecasts")

    for origin in origins:
        # Get training data (all data before origin)
        train = ts[ts.index < origin]

        # Skip if insufficient training data
        if len(train) < min_train:
            continue

        for horizon in horizons:
            # Target forecast date
            target_date = origin + pd.Timedelta(days=horizon - 1)

            # Skip if target date is beyond available data
            if target_date not in ts.index:
                pbar.update(1)
                continue

            # Get actual value
            actual = ts.loc[target_date]

            # SARIMA + Fourier
            try:
                pred_sarima = forecast_sarima_fourier(train, horizon)
                results['sarima_fourier'].append({
                    'date': target_date,
                    'origin': origin,
                    'horizon': horizon,
                    'model': 'SARIMA_Fourier',
                    'q0.1': pred_sarima['q0.1'],
                    'q0.5': pred_sarima['q0.5'],
                    'q0.9': pred_sarima['q0.9'],
                    'actual': actual
                })
            except Exception as e:
                pass  # Skip failed forecasts

            # LightGBM
            try:
                pred_lgb = forecast_lightgbm(train, horizon)
                results['lightgbm'].append({
                    'date': target_date,
                    'origin': origin,
                    'horizon': horizon,
                    'model': 'LightGBM',
                    'q0.1': pred_lgb['q0.1'],
                    'q0.5': pred_lgb['q0.5'],
                    'q0.9': pred_lgb['q0.9'],
                    'actual': actual
                })
            except Exception as e:
                pass

            # TabPFN-TS
            try:
                pred_tabpfn = forecast_tabpfn(train, horizon)
                results['tabpfn'].append({
                    'date': target_date,
                    'origin': origin,
                    'horizon': horizon,
                    'model': 'TabPFN_TS',
                    'q0.1': pred_tabpfn['q0.1'],
                    'q0.5': pred_tabpfn['q0.5'],
                    'q0.9': pred_tabpfn['q0.9'],
                    'actual': actual
                })
            except Exception as e:
                pass

            # Chronos-Tiny
            try:
                pred_chronos = forecast_chronos(train, horizon, chronos_pipeline)
                results['chronos'].append({
                    'date': target_date,
                    'origin': origin,
                    'horizon': horizon,
                    'model': 'Chronos_Tiny',
                    'q0.1': pred_chronos['q0.1'],
                    'q0.5': pred_chronos['q0.5'],
                    'q0.9': pred_chronos['q0.9'],
                    'actual': actual
                })
            except Exception as e:
                pass

            pbar.update(1)

    pbar.close()

    # Convert to DataFrames
    for model_name in results:
        results[model_name] = pd.DataFrame(results[model_name])

    return results

### Execute Rolling Forecasts

**Warning:** This cell will take several minutes to complete.

In [62]:
print("Starting rolling forecast generation...")
print(f"Total iterations: {len(ORIGINS) * len(HORIZONS)}")
print("\nThis will take approximately 4-5 minutes...\n")

forecast_results = run_rolling_forecasts(
    ts=ts,
    origins=ORIGINS,
    horizons=HORIZONS,
    min_train=MIN_TRAIN
)

print("\n✅ Rolling forecasts complete!")

Starting rolling forecast generation...
Total iterations: 48

This will take approximately 4-5 minutes...




Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 59.60it/s]













Processing: 100%|██████████| [00:01<00:00]

Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1509.83it/s]












Processing: 100%|██████████| [00:01<00:00]

Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1824.40it/s]












Processing: 100%|██████████| [00:01<00:00]

Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1691.25it/s]












Processing: 100%|██████████| [00:01<00:00]

Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 880.23it/s]













Processing: 100%|██████████| [00:01<00:00]

Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1402.31it/s]












Processing: 100%|██████████| [00:01<00:00]

Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1099.42it/s]












Processing: 100%|██████████| [00:01<00:00]

Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 938.74it/s]












Processing: 100%|████


✅ Rolling forecasts complete!


## 7. Save Results

In [63]:
# Save each model's forecasts as parquet
for model_name, df in forecast_results.items():
    output_path = results_dir / f"{model_name}.parquet"
    df.to_parquet(output_path, index=False)
    print(f"Saved {model_name}: {len(df)} forecasts → {output_path}")

print("\n✅ All forecast results saved!")

Saved sarima_fourier: 47 forecasts → /home/mikhailarutyunov/projects/time-series-flu/results/forecasts/sarima_fourier.parquet
Saved lightgbm: 47 forecasts → /home/mikhailarutyunov/projects/time-series-flu/results/forecasts/lightgbm.parquet
Saved tabpfn: 47 forecasts → /home/mikhailarutyunov/projects/time-series-flu/results/forecasts/tabpfn.parquet
Saved chronos: 47 forecasts → /home/mikhailarutyunov/projects/time-series-flu/results/forecasts/chronos.parquet

✅ All forecast results saved!


## 8. Summary Statistics

In [64]:
# Display summary of forecast counts by model and horizon
summary = []
for model_name, df in forecast_results.items():
    for horizon in HORIZONS:
        count = len(df[df['horizon'] == horizon])
        summary.append({
            'Model': model_name,
            'Horizon': horizon,
            'Forecasts': count
        })

summary_df = pd.DataFrame(summary)
summary_pivot = summary_df.pivot(index='Model', columns='Horizon', values='Forecasts')

print("\n" + "=" * 60)
print("FORECAST COUNT SUMMARY")
print("=" * 60)
print(summary_pivot)
print("\nTotal forecasts per model:")
print(summary_df.groupby('Model')['Forecasts'].sum())


FORECAST COUNT SUMMARY
Horizon         7   28
Model                 
chronos         24  23
lightgbm        24  23
sarima_fourier  24  23
tabpfn          24  23

Total forecasts per model:
Model
chronos           47
lightgbm          47
sarima_fourier    47
tabpfn            47
Name: Forecasts, dtype: int64


## Checkpoint Summary

**Expected outcomes:**
- 4 parquet files in `results/forecasts/` (one per model)
- Each file contains forecasts for all origins × horizons
- Columns: date, origin, horizon, model, q0.1, q0.5, q0.9, actual
- Total runtime: < 5 minutes
- No warnings about data leakage or CUDA

**Next:** Proceed to `03_evaluation.ipynb` for metrics and statistical tests.